### **Using Docker Containers**

This Notebook will show how to use scripts inside of docker containers. For a more complex way to use Containers go to the [snap processing notebook](snap_processing.ipynb).

**First some imports and global settings**

In [ ]:
from hera.workflows import Parameter, DAG, Workflow, Container
from hera.shared import global_config

global_config.host = "https://services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"

**Setting up container**

We need to use heras `Container` to set up which image to use. In the `Container` we can specify the commands to run the script. With `args` the arguments to pass to the function are set, `inputs` make it possible to set the arguments as an input parameter. In the image used for this example we have the function *write.py* printing the argument passed to it.

In [ ]:
echo = Container(name="write-message",
                image = "ghcr.io/oscipal/image_zarr:latest"
                command=["python", "write.py"],
                args=["{{inputs.parameters.message}}"],
                inputs=[Parameter(name="message")])

**Creating the Workflow**

As we have set inputs in the *echo* container to have the name "message" we can set the arguments with this name.

In [ ]:
with Workflow(
    generate_name="using-image-functions-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        msg1 = echo(name="msg1", arguments={"message": "Hello hera!"})
        msg2 = echo(name="msg2", arguments={"message": "Goodbye hera!"})

        msg1 >> msg2

**Submitting the Workflow**

In [ ]:
w.create()